In [ ]:
from tools_box import *
import pymbar
from pymbar import timeseries
from pymbar import MBAR


In [ ]:
stride = [10,100,1000]  # strides of data extraction
nstates = [10,30,50,100,200]  # number of clustered states
bin_min = 0.0      
bin_max = 0.8

In [ ]:
# let's reshape the u_kn file and delete duplicate ensembles

for i in range(len(stride)):
    ukn=np.load('../dataset/frames_%d/u_kn.npy'%(int(200000./stride[i])))
    new_ukn=[]
    for i in range(88):   # first 88 ensembles with harmonic restraints + lambdas (push+pull)
        new_ukn.append(ukn[i])
    for i in range(176,187):  # last 11 ensembles with lambdas only
        new_ukn.append(ukn[i])
    np.save('../dataset/frames_%d/sub_ukn.npy'%(int(200000./stride[i])),new_ukn)

In [ ]:
for i in range(len(stride)):
        for j in range(len(nstates)):

                dx = (bin_max-bin_min)/nstates[j]
                pmf_distances = np.arange(bin_min,bin_max+dx,dx)
                bincenters = (pmf_distances[0:-1] + pmf_distances[1:])/2.0
                print pmf_distances
                print bincenters
                u_kn = np.load('../dataset/frames_%d/sub_ukn.npy'%(int(200000./stride[i])))   # dimension: (# of thermodynamic states, number of thermodynamic states * number of frames for each ensemble)
                # number of ensembles = 11 * 8 + 11 = 99
                print u_kn.shape[1]
                distance = np.load('../dataset/frames_%d/distance_sub.npy'%(int(200000./stride[i])))  # raw distance file
                dis = []
                for k in range(len(distance)):
                        for l in distance[k]:
                                dis.append(l)
                dis = np.array(dis)
                reshape_dis = dis.reshape(1,u_kn.shape[1])  # reshape it to the dimension of 1*total frames of all ensembles
                K = u_kn.shape[0]    # K = 99 in this case
                nsnaps = distance.shape[-1]  # total number of frames
                N_k = []
                # number of frames of first 88 ensembles with harmonic restraint, pull + push yields 2 directions and duplicate thermodynamic ensembles and needs to be combined
                for m in range(88):
                        N_k.append((int(200000./stride[i])+1)*2)  
                # number of frames of last 11 ensembles with no harmonic restraint and lambda only
                for n in range(11):
                        N_k.append(int(200000./stride[i])+1)
                N_k = np.array(N_k)
                #print N_k
                #print len(N_k)
                print sum(N_k)   # total number of frames, should be equal to u_kn.shape[-1]
                u_kn -= u_kn.min()   
                bin_kn = np.zeros(reshape_dis.shape,np.int64)
                for p in range(reshape_dis.shape[1]):
                        bin_kn[0,p] = distance2bin(reshape_dis[0,p],bincenters)   # assign each frames based on 1D bins
                bin_kn = np.array(bin_kn[0])

                mbar = MBAR(u_kn, N_k, verbose=True)
                for q in range(u_kn.shape[0]):  # number of ensembles
                        print q
                        (f_i, df_i) = mbar.computePMF(u_kn[q], bin_kn, nstates[j])
                        print f_i, df_i
                        fout = open('states_%d/frames_%d/mbar_%d_sub.dat'%(nstates[j],int(200000./stride[i]),q),'w')
                        fout.write('# distance (nm)\tf (kT)\tdf (kT)\n')
                        for r in range(nstates[j]):
                                fout.write("%8.4f\t%8.4f\t%8.4f\n" % (bincenters[r], f_i[r], df_i[r]) )
                        fout.close()
                        print 'Wrote'
